<a href="https://colab.research.google.com/github/levhyun/MachineLearningInSchool/blob/main/2023.04.19/2023_04_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

rnwkgus_Books = pd.read_csv('rnwkgusBook.csv', encoding='EUC-KR', low_memory=False) # csv 파일을 데이터프레임으로 읽어오기

In [57]:
def dataCleaning(rnwkgus_Books):
  # NaN인 열 삭제
  rnwkgus_Books = rnwkgus_Books.dropna(axis=1, how='all')

  # 대출건수를 합치기 위해 필요한 행만 추출하여 새로운 데이터프레임을 만든다.
  count = rnwkgus_Books[['도서명','저자','ISBN','권','대출건수']]

  # 도서명, 저자, ISBN, 권을 기준으로 대출건수를 그룹한다.
  loacCount = count.groupby(by=['도서명','ISBN','권'],dropna=False).sum() 
  
  # 도서명, 저자, ISBN, 권을 기준으로 중복된 행을 True로 표기한다.
  dupRows = rnwkgus_Books.duplicated(subset=['도서명','저자','ISBN','권'])

  # 불리언 배열을 반전시켜 고유한 행을 True로 표시하다.
  uniqueRows = ~dupRows

  # 고유한 행을 새로운 데이터프레임으로 만든다.
  rnwkgus_Books2 = rnwkgus_Books[uniqueRows]

  # 도서명, 저자, ISBN, 권을 인덱스로 지정한다.
  rnwkgus_Books2.set_index(['도서명','저자','ISBN','권'],inplace=True)

  # 원본 데이터프레임을 업데이트
  rnwkgus_Books2.update(loacCount)

  # 데이터프레임 인덱스를 재설정
  rnwkgus_Books3 = rnwkgus_Books2.reset_index()

  # 원래 열을 순서대로 맟춘다
  rnwkgus_Books3 = rnwkgus_Books3[rnwkgus_Books.columns]

  # 결과를 반환한다.
  return rnwkgus_Books3

In [52]:
rnwkgus_Books3 = dataCleaning(rnwkgus_Books)
rnwkgus_Books3.head()

<ipython-input-51-596e2b4d2c98>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  loacCount = count.groupby(by=['도서명','ISBN','권'],dropna=False).sum()


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,헤어질 결심 각본,"정서경,박찬욱 지음",을유문화사,2022,9788932474755,NaN,0,NaN,812.66,1,0,2023-03-10,NaN
1,2,화이트 러시,히가시노 게이고 지음;민경욱 옮김,㈜소미미디어,2023,9791138415477,NaN,0,NaN,833.60,1,1,2023-03-10,NaN
2,3,크리스마스 타일,김금희 지음,창비,2022,9788936438890,NaN,0,NaN,813.60,1,1,2023-03-10,NaN
3,4,체셔 크로싱,앤디 위어 글;사라 앤더슨 그림;황석희 옮김,알에이치코리아(RHK),2022,9788925577319,NaN,0,NaN,843.60,1,0,2023-03-10,NaN
4,5,자발적 방관육아,최은아 지음,쌤앤파커스,2023,9791165346805,NaN,0,NaN,598.10,1,1,2023-03-10,NaN


In [40]:
def getBookInfo(row):
  title = row['도서명']
  author = row['저자']
  pub = row['출판사']
  year = row['발행년도']
  isbn = row['ISBN']
  # Yes24 도서 검색 페이지 URL을 포맷팅한다.
  url = f'http://www.yes24.com/Product/Search?domain=BOOK&query={isbn}' 

  # URL 페이지의 HTML을 가져온다.
  r = requests.get(url)

  # 가져온 HTML을 파싱한다.
  html = BeautifulSoup(r.text, 'html.parser')

  # title이 NaN이면 HTML에서 title 값을 찾아 채워 넣는다.
  try:
    if pd.isna(title):
      title = html.find('a', attrs={'class':'gd_name'}).get_text()
  except AttributeError:
    pass

  # author이 NaN이면 HTML에서 author 값을 찾아 채워 넣는다.
  try:
    if pd.isna(author):
      authors = html.find('span', attrs={'class':'info_auth'}).find_all('a') 
      author = list(tag.get_text() for tag in authors) 
  except AttributeError:
    pass

  # pub이 NaN이면 HTML에서 pub 값을 찾아 채워 넣는다.
  try:
    if pd.isna(pub):
      pub = html.find('span', attrs={'class':'info_pub'}).find('a').get_text()
  except AttributeError:
    pass

  # year이 -1이면 HTML에서 year 값을 찾아 채워 넣는다.
  try:
    if year == -1:
      year_str = html.find('span', attrs={'class':'info_date'}).get_text()
      year = re.findall(r'\d{4}', year_str)[0]
  except AttributeError:
    pass

  # 결과를 반환한다.
  return title,author,pub,year

In [53]:
def dataFixing(rnwkgus_Books3):

  # 도서권수와 대출건수를 int32로 변환한다. 
  rnwkgus_Books3 = rnwkgus_Books3.astype({'도서권수':'int32', '대출건수':'int32'})

  # NaN인 세트 ISBN을 빈 문자열로 변환한다.
  set_isbn_na_rows = rnwkgus_Books3['세트 ISBN'].isna()
  rnwkgus_Books3.loc[set_isbn_na_rows, '세트 ISBN'] = ''

  # 발행년도 열에서 연도 네자리를 추출하여 대체한다.
  rnwkgus_Books4 = rnwkgus_Books3.replace({'발행년도':{r'.*(\d{4}).*'}}, r'\1', regex=True)
  unkown_year = rnwkgus_Books4['발행년도'].str.contains('\D', na=True)

  # 나머지 발행년도는 -1로 변환한다.
  rnwkgus_Books4.loc[unkown_year, '발행년도'] = '-1'

  # 발행년도를 int32로 변환한다.
  rnwkgus_Books4 = rnwkgus_Books4.astype({'발행년도': 'int32'})

  # 발행년도가 4000년 이상인 경우 2333년을 뺀다
  dangun_yy_rows = rnwkgus_Books4['발행년도'].gt(4000)
  rnwkgus_Books4.loc[dangun_yy_rows, '발행년도'] = rnwkgus_Books4.loc[dangun_yy_rows, '발행년도'] - 2333

  # 여전히 발행년도가 4000년 이상인 경우 -1로 변환한다.
  dangun_year = rnwkgus_Books4['발행년도'].gt(4000)
  rnwkgus_Books4.loc[dangun_year, '발행년도'] = -1

  # 발행년도가 0 ~ 1900년 사이인 경우 발행년도를 -1로 변환한다.
  old_books = rnwkgus_Books4['발행년도'].gt(0) & rnwkgus_Books4['발행년도'].lt(1900)
  rnwkgus_Books4.loc[old_books, '발행년도'] = -1
  
  # 도서명, 저자, 출판사가 NaN이거나 발행년도가 -1인 행을 추출한다.
  na_rows = rnwkgus_Books4['도서명'].isna() | rnwkgus_Books4['저자'].isna() | rnwkgus_Books4['출판사'].isna() | rnwkgus_Books4['발행년도'].eq(-1)

  # Yes24 도서 상세 페이지에서 누락된 정보를 채운다.
  updated_sample = rnwkgus_Books4[na_rows].apply(getBookInfo, axis=1, result_type='expand')

  # 칼럼명을 도서명, 저자, 출판사, 발행년도로 변경한다
  updated_sample.columns = ['도서명', '저자', '출판사', '발행년도']

  # ns_book5 데이터 프레임을 업데이트한다.
  rnwkgus_Books4.update(updated_sample, overwrite=True)

  # 도서명, 저자, 출판사가 NaN이거나 발행년도가 -1인 행을 삭제한다.
  rnwkgus_Books5 = rnwkgus_Books4.dropna(subset=['도서명', '저자', '출판사'])
  rnwkgus_Books5 = rnwkgus_Books5[rnwkgus_Books5['발행년도'] != -1]

  return rnwkgus_Books5 # 결과를 반환한다.

In [55]:
rnwkgus_Books6 = dataFixing(rnwkgus_Books3)
rnwkgus_Books6.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,헤어질 결심 각본,"정서경,박찬욱 지음",을유문화사,2022,9788932474755,,0,NaN,812.66,1,0,2023-03-10,NaN
1,2,화이트 러시,히가시노 게이고 지음;민경욱 옮김,㈜소미미디어,2023,9791138415477,,0,NaN,833.60,1,1,2023-03-10,NaN
2,3,크리스마스 타일,김금희 지음,창비,2022,9788936438890,,0,NaN,813.60,1,1,2023-03-10,NaN
3,4,체셔 크로싱,앤디 위어 글;사라 앤더슨 그림;황석희 옮김,알에이치코리아(RHK),2022,9788925577319,,0,NaN,843.60,1,0,2023-03-10,NaN
4,5,자발적 방관육아,최은아 지음,쌤앤파커스,2023,9791165346805,,0,NaN,598.10,1,1,2023-03-10,NaN


In [58]:
def dataCleaning(rnwkgus_Books):
  # NaN인 열 삭제
  rnwkgus_Books = rnwkgus_Books.dropna(axis=1, how='all')

  # 대출건수를 합치기 위해 필요한 행만 추출하여 새로운 데이터프레임을 만든다.
  count = rnwkgus_Books[['도서명','저자','ISBN','권','대출건수']]

  # 도서명, 저자, ISBN, 권을 기준으로 대출건수를 그룹한다.
  loacCount = count.groupby(by=['도서명','ISBN','권'],dropna=False).sum() 
  
  # 도서명, 저자, ISBN, 권을 기준으로 중복된 행을 True로 표기한다.
  dupRows = rnwkgus_Books.duplicated(subset=['도서명','저자','ISBN','권'])

  # 불리언 배열을 반전시켜 고유한 행을 True로 표시하다.
  uniqueRows = ~dupRows

  # 고유한 행을 새로운 데이터프레임으로 만든다.
  rnwkgus_Books2 = rnwkgus_Books[uniqueRows]

  # 도서명, 저자, ISBN, 권을 인덱스로 지정한다.
  rnwkgus_Books2.set_index(['도서명','저자','ISBN','권'],inplace=True)

  # 원본 데이터프레임을 업데이트
  rnwkgus_Books2.update(loacCount)

  # 데이터프레임 인덱스를 재설정
  rnwkgus_Books3 = rnwkgus_Books2.reset_index()

  # 원래 열을 순서대로 맟춘다
  rnwkgus_Books3 = rnwkgus_Books3[rnwkgus_Books.columns]

  # 결과를 반환한다.
  return rnwkgus_Books3

def getBookInfo(row):
  title = row['도서명']
  author = row['저자']
  pub = row['출판사']
  year = row['발행년도']
  isbn = row['ISBN']
  # Yes24 도서 검색 페이지 URL을 포맷팅한다.
  url = f'http://www.yes24.com/Product/Search?domain=BOOK&query={isbn}' 

  # URL 페이지의 HTML을 가져온다.
  r = requests.get(url)

  # 가져온 HTML을 파싱한다.
  html = BeautifulSoup(r.text, 'html.parser')

  # title이 NaN이면 HTML에서 title 값을 찾아 채워 넣는다.
  try:
    if pd.isna(title):
      title = html.find('a', attrs={'class':'gd_name'}).get_text()
  except AttributeError:
    pass

  # author이 NaN이면 HTML에서 author 값을 찾아 채워 넣는다.
  try:
    if pd.isna(author):
      authors = html.find('span', attrs={'class':'info_auth'}).find_all('a') 
      author = list(tag.get_text() for tag in authors) 
  except AttributeError:
    pass

  # pub이 NaN이면 HTML에서 pub 값을 찾아 채워 넣는다.
  try:
    if pd.isna(pub):
      pub = html.find('span', attrs={'class':'info_pub'}).find('a').get_text()
  except AttributeError:
    pass

  # year이 -1이면 HTML에서 year 값을 찾아 채워 넣는다.
  try:
    if year == -1:
      year_str = html.find('span', attrs={'class':'info_date'}).get_text()
      year = re.findall(r'\d{4}', year_str)[0]
  except AttributeError:
    pass

  # 결과를 반환한다.
  return title,author,pub,year

def dataFixing(rnwkgus_Books3):

  # 도서권수와 대출건수를 int32로 변환한다. 
  rnwkgus_Books3 = rnwkgus_Books3.astype({'도서권수':'int32', '대출건수':'int32'})

  # NaN인 세트 ISBN을 빈 문자열로 변환한다.
  set_isbn_na_rows = rnwkgus_Books3['세트 ISBN'].isna()
  rnwkgus_Books3.loc[set_isbn_na_rows, '세트 ISBN'] = ''

  # 발행년도 열에서 연도 네자리를 추출하여 대체한다.
  rnwkgus_Books4 = rnwkgus_Books3.replace({'발행년도':{r'.*(\d{4}).*'}}, r'\1', regex=True)
  unkown_year = rnwkgus_Books4['발행년도'].str.contains('\D', na=True)

  # 나머지 발행년도는 -1로 변환한다.
  rnwkgus_Books4.loc[unkown_year, '발행년도'] = '-1'

  # 발행년도를 int32로 변환한다.
  rnwkgus_Books4 = rnwkgus_Books4.astype({'발행년도': 'int32'})

  # 발행년도가 4000년 이상인 경우 2333년을 뺀다
  dangun_yy_rows = rnwkgus_Books4['발행년도'].gt(4000)
  rnwkgus_Books4.loc[dangun_yy_rows, '발행년도'] = rnwkgus_Books4.loc[dangun_yy_rows, '발행년도'] - 2333

  # 여전히 발행년도가 4000년 이상인 경우 -1로 변환한다.
  dangun_year = rnwkgus_Books4['발행년도'].gt(4000)
  rnwkgus_Books4.loc[dangun_year, '발행년도'] = -1

  # 발행년도가 0 ~ 1900년 사이인 경우 발행년도를 -1로 변환한다.
  old_books = rnwkgus_Books4['발행년도'].gt(0) & rnwkgus_Books4['발행년도'].lt(1900)
  rnwkgus_Books4.loc[old_books, '발행년도'] = -1
  
  # 도서명, 저자, 출판사가 NaN이거나 발행년도가 -1인 행을 추출한다.
  na_rows = rnwkgus_Books4['도서명'].isna() | rnwkgus_Books4['저자'].isna() | rnwkgus_Books4['출판사'].isna() | rnwkgus_Books4['발행년도'].eq(-1)

  # Yes24 도서 상세 페이지에서 누락된 정보를 채운다.
  updated_sample = rnwkgus_Books4[na_rows].apply(getBookInfo, axis=1, result_type='expand')

  # 칼럼명을 도서명, 저자, 출판사, 발행년도로 변경한다
  updated_sample.columns = ['도서명', '저자', '출판사', '발행년도']

  # ns_book5 데이터 프레임을 업데이트한다.
  rnwkgus_Books4.update(updated_sample, overwrite=True)

  # 도서명, 저자, 출판사가 NaN이거나 발행년도가 -1인 행을 삭제한다.
  rnwkgus_Books5 = rnwkgus_Books4.dropna(subset=['도서명', '저자', '출판사'])
  rnwkgus_Books5 = rnwkgus_Books5[rnwkgus_Books5['발행년도'] != -1]

  return rnwkgus_Books5 # 결과를 반환한다.

def importList():
  import pandas as pd
  import numpy as np
  import requests
  from bs4 import BeautifulSoup
  import re

def final_BookDataAnalysis(fileName):
  # 필요한 라이브러리를 임포트한다.
  importList()

  # csv 파일을 데이터프레임으로 읽어온다.
  rnwkgus_Books = pd.read_csv(f'{fileName}', encoding='EUC-KR', low_memory=False)
  
  # dataCleaning() 함수로 데이터를 전처리한다.
  rnwkgus_Books3 = dataCleaning(rnwkgus_Books)

  # dataFixing() 함수로 잘못된 값을 수정하거나 NaN을 채운다.
  rnwkgus_Books6 = dataFixing(rnwkgus_Books3)

  # 최종 결과를 반환한다.
  return rnwkgus_Books6

# final_BookDataAnalysis() 함수로 데이터를 분석한 결과를 반환받는다.
finalData = final_BookDataAnalysis('rnwkgusBook.csv')

# 최종 데이터 프레임 출력
finalData

<ipython-input-58-037f4bdbc739>:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  loacCount = count.groupby(by=['도서명','ISBN','권'],dropna=False).sum()


,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자
0,1,헤어질 결심 각본,"정서경,박찬욱 지음",을유문화사,2022,9788932474755,,0,NaN,812.66,1,0,2023-03-10
1,2,화이트 러시,히가시노 게이고 지음;민경욱 옮김,㈜소미미디어,2023,9791138415477,,0,NaN,833.60,1,1,2023-03-10
2,3,크리스마스 타일,김금희 지음,창비,2022,9788936438890,,0,NaN,813.60,1,1,2023-03-10
3,4,체셔 크로싱,앤디 위어 글;사라 앤더슨 그림;황석희 옮김,알에이치코리아(RHK),2022,9788925577319,,0,NaN,843.60,1,0,2023-03-10
4,5,자발적 방관육아,최은아 지음,쌤앤파커스,2023,9791165346805,,0,NaN,598.10,1,1,2023-03-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14647,14776,(고정관념을 날려버리는) 5분 철학 오프너,줄리아 드 퓌네스 지음;이나무 옮김,이숲,2012,9788994228150,,0,NaN,104.00,1,1,2014-02-11
14648,14777,(겸손의 미덕으로 미래를 바꾼) 후진타오 이야기,박근형 지음,명진출판,2012,9788976776235,,0,8,990.80,1,0,2014-02-11
14649,14778,앤디 워홀 이야기,아서 단토 지음;박선령 옮김;이혜경 엮음,명진출판,2012,9788976776280,,0,10,990.80,1,0,2014-02-11
14650,14779,4001,신정아 지음,사월의책,2011,9788996461067,,0,NaN,818.00,1,2,2014-02-11
